In [1]:
import numpy as np
import plost
import requests
import streamlit as st
import os
import random
from scapy.all import rdpcap
import collections
import tempfile
import sys
import pandas as pd
from scapy.utils import corrupt_bytes
from streamlit_echarts import st_echarts
import geoip2.database
import pydeck as pdk
import folium

# from scapy.layers.inet import IP,TCP,UDP,
from utils.pcap_decode import PcapDecode
import time

In [26]:
def get_all_pcap(PCAPS, PD):
    pcaps = collections.OrderedDict()
    for count, i in enumerate(PCAPS, 1):
        pcaps[count] = PD.ether_decode(i)
    return pcaps

def process_json_data(json_data):
    # Convert JSON data to a pandas DataFrame
    df = pd.DataFrame.from_dict(json_data, orient='index')
    return df

def calculate_live_time(pcap_data):
    timestamps = [float(packet.time) for packet in pcap_data]  # Convert to float
    start_time = min(timestamps)
    end_time = max(timestamps)
    live_time_duration = end_time - start_time
    live_time_duration_str = str(pd.Timedelta(seconds=live_time_duration))
    return start_time, end_time, live_time_duration, live_time_duration_str

In [27]:
file_path = "sample.pcap"  # Change this to the actual file path
uploaded_file = rdpcap(file_path)

if uploaded_file is not None:
    # Process the uploaded PCAP file
    pcap_data = uploaded_file

    # Example: Get all PCAPs
    start_time, end_time, live_time_duration, live_time_duration_str = calculate_live_time(pcap_data)
    all_data = get_all_pcap(pcap_data, PcapDecode())
    dataframe_data = process_json_data(all_data)


     # Add live time information to the data frame
    dataframe_data['Start Time'] = start_time
    dataframe_data['End Time'] = end_time
    dataframe_data['Live Time Duration'] = live_time_duration_str


    
    print("All PCAPs:")

dataframe_data

All PCAPs:


,time,Source,Destination,len,info,Procotol,Start Time,End Time,Live Time Duration
1,2022-11-23 18:08:16,172.21.224.2:22,35.235.244.34:35193,210,Ether / IP / TCP 172.21.224.2:ssh > 35.235.244...,SSH,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
2,2022-11-23 18:08:16,35.235.244.34:35193,172.21.224.2:22,66,Ether / IP / TCP 35.235.244.34:35193 > 172.21....,SSH,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
3,2022-11-23 18:08:24,35.235.244.34:35193,172.21.224.2:22,162,Ether / IP / TCP 35.235.244.34:35193 > 172.21....,SSH,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
4,2022-11-23 18:08:24,172.21.224.2:22,35.235.244.34:35193,162,Ether / IP / TCP 172.21.224.2:ssh > 35.235.244...,SSH,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
5,2022-11-23 18:08:24,35.235.244.34:35193,172.21.224.2:22,66,Ether / IP / TCP 35.235.244.34:35193 > 172.21....,SSH,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
...,...,...,...,...,...,...,...,...,...
196,2022-11-23 18:08:59,216.239.32.27:443,172.21.224.2:57724,1474,Ether / IP / TCP 216.239.32.27:https > 172.21....,HTTPS,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
197,2022-11-23 18:08:59,216.239.32.27:443,172.21.224.2:57724,1474,Ether / IP / TCP 216.239.32.27:https > 172.21....,HTTPS,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
198,2022-11-23 18:08:59,172.21.224.2:57724,216.239.32.27:443,66,Ether / IP / TCP 172.21.224.2:57724 > 216.239....,HTTPS,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994
199,2022-11-23 18:08:59,216.239.32.27:443,172.21.224.2:57724,2882,Ether / IP / TCP 216.239.32.27:https > 172.21....,HTTPS,1.669207e+09,1.669207e+09,0 days 00:00:42.548099994


In [28]:
dataframe_data.columns

Index(['time', 'Source', 'Destination', 'len', 'info', 'Procotol',
       'Start Time', 'End Time', 'Live Time Duration'],
      dtype='object')

In [29]:
dataframe_data['time']

1      2022-11-23 18:08:16
2      2022-11-23 18:08:16
3      2022-11-23 18:08:24
4      2022-11-23 18:08:24
5      2022-11-23 18:08:24
              ...         
196    2022-11-23 18:08:59
197    2022-11-23 18:08:59
198    2022-11-23 18:08:59
199    2022-11-23 18:08:59
200    2022-11-23 18:08:59
Name: time, Length: 200, dtype: object

In [1]:
from scapy.all import *
data = "sample.pcap"
a = rdpcap(data)
os.system("tshark -r Eavesdrop_Data.pcap -Y http -w Eavesdrop_Data_http.pcap")
sessions = a.sessions()
i = 1
for session in sessions:
    http_payload = ""
    for packet in sessions[session]:
        try:
            if packet[TCP].dport == 80 or packet[TCP].sport == 80:
                print packet[TCP].payload
        except:
            pass

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (989439830.py, line 12)